# Introduction

![](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b7/Woodstock_poster.jpg/800px-Woodstock_poster.jpg)

# Setup

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
dfs = pd.read_html("https://en.wikipedia.org/wiki/Woodstock")

# Creating the dataframe 

In [ ]:
fridayToSaturday = dfs[1]

fridayToSaturday["Day"] = "Friday, August 15 – Saturday, August 16"

fridayToSaturday

In [ ]:
saturdayToSunday = dfs[2]

saturdayToSunday["Day"] = "Saturday, August 16 – Sunday, August 17"

saturdayToSunday

In [ ]:
sundayToMonday = dfs[3]

sundayToMonday["Day"] = "Sunday, August 17 – Monday, August 18"

sundayToMonday

In [ ]:
df = pd.concat([fridayToSaturday, saturdayToSunday, sundayToMonday], ignore_index=True)

df

# Data cleaning and transformation

In [ ]:
df['Time'] = df['Time'].str.replace('Midnight', '12:00 AM')

In [ ]:
# Splitting the time column
df[['Start Time', 'End Time']] = df['Time'].str.split(' – ', expand=True)

# Convert starting and ending time columns to datetime objects
df['Start Time'] = pd.to_datetime(df['Start Time'], format='%I:%M %p')
df['End Time'] = pd.to_datetime(df['End Time'], format='%I:%M %p')

# Calculate concert duration
df['Concert Duration'] = df['End Time'] - df['Start Time']

# Format the datetime columns to show only the time
df['Start Time'] = df['Start Time'].dt.strftime('%I:%M %p')
df['End Time'] = df['End Time'].dt.strftime('%I:%M %p')

# Convert "Concert Duration" to string
df['Concert Duration'] = df['Concert Duration'].astype(str)

# Extract hours and minutes from "Concert Duration" and convert to a consistent format
df['Concert Duration'] = df['Concert Duration'].str.extract(r'(\d+:\d+)')

# Function to convert HH:MM to minutes
def convert_to_minutes(time_str):
    hours, minutes = map(int, time_str.split(':'))
    return hours * 60 + minutes

# Apply the function to the 'Concert Duration' column
df['Concert Duration (minutes)'] = df['Concert Duration'].apply(convert_to_minutes)

In [ ]:
df

In [ ]:
plt.figure(figsize=(10, 8))

sns.barplot(df, y="Artist", x="Concert Duration (minutes)", orient="h")